# Back-Testing Piotroski's F-SCORE Strategy

## Import Libraries

In [1]:
# import necessary libraries
import wrds
import pandas as pd
import datetime as dt
import numpy as np
from datetime import datetime
from datetime import timedelta
from calendar import isleap
from workalendar.usa import UnitedStates
from pandas_datareader import data

# note: wrds provides an api to wharton research data services which provides access to various databases
# note: compustat can be accessed through wrds
# note: the calender library will be used to identify leap years
# note: the workalendar.usa library will be used to identify days that are public holidays in the US

# connect to wrds
db = wrds.Connection(wrds_username='acka662')

# set US calender
cal = UnitedStates()

Loading library list...
Done


## Read Dataset to Pandas Dataframe

In [2]:
# read fundamentals_fscore.csv file into a pandas dataframe
funda_df = pd.read_csv('fundamentals_fscore.csv')
# display funda_df
display(funda_df.head())

,CIK,Ticker_Symbol,Company_Name,Sector,Data_Date,Market_Capitalization,Total_Assets,Total_Assets_PY1,Total_Assets_PY2,Total_Liabilities,...,F_CLEVER,CLIQUID,F_CLIQUID,EQ_OFFER,F_EQ_OFFER,CMARGIN,F_CMARGIN,CTURN,F_CTURN,F_SCORE
0,1750,AIR,AAR CORP,Capital Goods,2011-05-31,1049.8206,1703.727,1501.042,1377.511,868.438,...,1,-0.455190,0,0.297,0,-0.004629,0,0.201443,1,6
1,1750,AIR,AAR CORP,Capital Goods,2012-05-31,485.2897,2195.653,1703.727,1501.042,1329.631,...,0,0.049832,1,0.492,0,-0.008424,0,0.034591,1,5
2,1750,AIR,AAR CORP,Capital Goods,2013-05-31,790.0029,2136.900,2195.653,1703.727,1217.400,...,1,0.410468,1,-0.891,1,0.010205,1,-0.230628,0,7
3,1750,AIR,AAR CORP,Capital Goods,2014-05-31,961.3080,2199.500,2136.900,2195.653,1198.800,...,1,0.120341,1,0.178,0,0.014049,1,-0.034682,0,7
4,1750,AIR,AAR CORP,Capital Goods,2015-05-31,1046.3954,1515.000,2199.500,2136.900,669.900,...,1,-0.461891,0,-4.137,1,-0.065087,0,-0.227468,0,3


In [3]:
# note: the cik of each company is usuallly 10 digits in length
# note: the leading 0's from some ciks have been removed
# note: therefore, these leading 0's have to be added again to ensure all ciks are of length 10

# add leading 0's to ciks that have a length less than 10
funda_df['CIK'] = funda_df['CIK'].apply(lambda x: '{0:0>10}'.format(x))

# convert values in the Data_Date column of funda_df to datetime 
funda_df['Data_Date'] = pd.to_datetime(funda_df['Data_Date'])

## COMPUSTAT - Stock Price Data Extraction

In [4]:
pd.options.mode.chained_assignment = None

# note: in order to back-test portfolio performance, stock price data is required
# note: pricing data for each observation in funda_df will be obtained
# note: each observation will have pricing data that corresponds to the date on which the financial
#       statement data is avaliable to the public (this is the final date variable in the funda table on compustat)
# note: the sell date of each observation will be one year from the final date described in the note above
# note: the pricing data for more recent observations (which the fiscal year has not concluded) will not be
#       available for a year - these observations will be sold on 02/09/2019

# define a dataframe that will store the pricing data for each observation in funda_df
# note: this dataframe will also store the observations corresponding F-SCORE
stock_price_df = funda_df[['Ticker_Symbol', 'Company_Name', 'Data_Date', 'BM_Quintile', 'F_SCORE']]
# filter very high b/m companies from stock_price_df
stock_price_df = stock_price_df.loc[stock_price_df['BM_Quintile']=='Very High'].reset_index(drop=True)

# create column in stock_price_df to store purchase date
# note: purchase date will be equal to the final date variable in the funda table in the compustat database
stock_price_df['Purchase_Date'] = 0
# retrieve final data from funda table
for idx, row in stock_price_df.iterrows():
    purchase_date = db.raw_sql("""SELECT DISTINCT fdate
                                  FROM compa.funda WHERE tic=%s
                                  AND datadate=%s""", params=(row['Ticker_Symbol'], row['Data_Date']))
    # provide user feedback
    print('-----------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    if purchase_date.shape[0]==0:
        stock_price_df.loc[idx, 'Purchase_Date'] = 'N/A'
    else:
        stock_price_df.loc[idx, 'Purchase_Date'] = purchase_date['fdate'][0]
    print('data available to public on:', stock_price_df.loc[idx, 'Purchase_Date'])
    print('\n')

-----------------------------------------------------------------------------
0 AIR
data available to public on: 2012-07-23


-----------------------------------------------------------------------------
1 AIR
data available to public on: 2013-08-01


-----------------------------------------------------------------------------
2 AIR
data available to public on: 2014-07-20


-----------------------------------------------------------------------------
3 AIR
data available to public on: 2016-07-18


-----------------------------------------------------------------------------
4 AIR
data available to public on: N/A


-----------------------------------------------------------------------------
5 HES
data available to public on: 2012-02-28


-----------------------------------------------------------------------------
6 HES
data available to public on: 2013-03-03


-----------------------------------------------------------------------------
7 HES
data available to public on: 2014-03-04



-----------------------------------------------------------------------------
67 CTL
data available to public on: 2018-03-06


-----------------------------------------------------------------------------
68 CTL
data available to public on: 2019-03-13


-----------------------------------------------------------------------------
69 CSS
data available to public on: 2011-06-08


-----------------------------------------------------------------------------
70 CSS
data available to public on: 2012-06-08


-----------------------------------------------------------------------------
71 CSS
data available to public on: 2013-06-06


-----------------------------------------------------------------------------
72 CSS
data available to public on: 2014-06-04


-----------------------------------------------------------------------------
73 CSS
data available to public on: 2015-06-09


-----------------------------------------------------------------------------
74 CSS
data available to public o

-----------------------------------------------------------------------------
133 VHI
data available to public on: 2016-03-24


-----------------------------------------------------------------------------
134 VHI
data available to public on: 2019-03-28


-----------------------------------------------------------------------------
135 LPX
data available to public on: 2012-03-18


-----------------------------------------------------------------------------
136 LXU
data available to public on: 2016-03-22


-----------------------------------------------------------------------------
137 LXU
data available to public on: 2017-03-15


-----------------------------------------------------------------------------
138 LXU
data available to public on: 2018-03-20


-----------------------------------------------------------------------------
139 LXU
data available to public on: 2019-03-18


-----------------------------------------------------------------------------
140 LDL
data available to 

-----------------------------------------------------------------------------
197 SCX
data available to public on: 2018-09-14


-----------------------------------------------------------------------------
198 SCX
data available to public on: N/A


-----------------------------------------------------------------------------
199 SUP
data available to public on: 2012-03-20


-----------------------------------------------------------------------------
200 SUP
data available to public on: 2013-03-27


-----------------------------------------------------------------------------
201 SUP
data available to public on: 2014-03-27


-----------------------------------------------------------------------------
202 SUP
data available to public on: 2015-03-27


-----------------------------------------------------------------------------
203 SUP
data available to public on: 2016-04-01


-----------------------------------------------------------------------------
204 SUP
data available to public 

-----------------------------------------------------------------------------
263 HVT
data available to public on: 2012-03-22


-----------------------------------------------------------------------------
264 PIR
data available to public on: 2018-05-25


-----------------------------------------------------------------------------
265 SU
data available to public on: 2012-03-19


-----------------------------------------------------------------------------
266 MUX
data available to public on: 2014-04-10


-----------------------------------------------------------------------------
267 MUX
data available to public on: 2015-04-09


-----------------------------------------------------------------------------
268 MUX
data available to public on: 2016-04-05


-----------------------------------------------------------------------------
269 VAL
data available to public on: N/A


-----------------------------------------------------------------------------
270 VAL
data available to public o

...

-----------------------------------------------------------------------------
1177 STNG
data available to public on: 2017-04-07


-----------------------------------------------------------------------------
1178 STNG
data available to public on: 2018-04-20


-----------------------------------------------------------------------------
1179 STNG
data available to public on: 2019-04-15


-----------------------------------------------------------------------------
1180 TOWR
data available to public on: 2013-03-22


-----------------------------------------------------------------------------
1181 OAS
data available to public on: 2015-03-13


-----------------------------------------------------------------------------
1182 OAS
data available to public on: 2016-03-21


-----------------------------------------------------------------------------
1183 OAS
data available to public on: 2018-03-29


-----------------------------------------------------------------------------
1184 OAS
data a

-----------------------------------------------------------------------------
1243 TROX
data available to public on: 2017-03-23


-----------------------------------------------------------------------------
1244 TROX
data available to public on: 2019-03-14


-----------------------------------------------------------------------------
1245 POST
data available to public on: 2014-12-04


-----------------------------------------------------------------------------
1246 EROS
data available to public on: 2016-07-28


-----------------------------------------------------------------------------
1247 EROS
data available to public on: 2017-08-02


-----------------------------------------------------------------------------
1248 EROS
data available to public on: 2018-08-23


-----------------------------------------------------------------------------
1249 EROS
data available to public on: N/A


-----------------------------------------------------------------------------
1250 BEP
data avail

-----------------------------------------------------------------------------
1307 DLNG
data available to public on: 2018-05-01


-----------------------------------------------------------------------------
1308 DLNG
data available to public on: 2019-06-11


-----------------------------------------------------------------------------
1309 AGN
data available to public on: 2017-02-28


-----------------------------------------------------------------------------
1310 AGN
data available to public on: 2018-02-19


-----------------------------------------------------------------------------
1311 AGN
data available to public on: 2019-02-20


-----------------------------------------------------------------------------
1312 VNCE
data available to public on: 2018-06-23


-----------------------------------------------------------------------------
1313 NEWM
data available to public on: 2017-03-10


-----------------------------------------------------------------------------
1314 NEWM
data 

-----------------------------------------------------------------------------
1372 EURN
data available to public on: 2018-05-23


-----------------------------------------------------------------------------
1373 EURN
data available to public on: 2019-04-30


-----------------------------------------------------------------------------
1374 WLKP
data available to public on: 2017-04-13


-----------------------------------------------------------------------------
1375 WLKP
data available to public on: 2018-04-24


-----------------------------------------------------------------------------
1376 WLKP
data available to public on: 2019-04-19


-----------------------------------------------------------------------------
1377 FCAU
data available to public on: 2019-03-30


-----------------------------------------------------------------------------
1378 ATTO
data available to public on: 2019-04-16


-----------------------------------------------------------------------------
1379 VSLR
da

-----------------------------------------------------------------------------
1436 BH
data available to public on: 2011-12-14


-----------------------------------------------------------------------------
1437 BH
data available to public on: 2012-12-11


-----------------------------------------------------------------------------
1438 BH
data available to public on: 2013-12-11


-----------------------------------------------------------------------------
1439 BH
data available to public on: 2014-12-02


-----------------------------------------------------------------------------
1440 BH
data available to public on: 2016-03-02


-----------------------------------------------------------------------------
1441 BH
data available to public on: 2017-03-13


-----------------------------------------------------------------------------
1442 BH
data available to public on: 2018-03-19


-----------------------------------------------------------------------------
1443 BH
data available to 

In [5]:
# investigate 'N/A' purchase date observations in stock_price_df
stock_price_df.query('Purchase_Date=="N/A"')

,Ticker_Symbol,Company_Name,Data_Date,BM_Quintile,F_SCORE,Purchase_Date
4,AIR,AAR CORP,2019-05-31,Very High,5,N/A
44,BGG,BRIGGS & STRATTON,2019-06-30,Very High,3,N/A
132,LCI,LANNETT CO INC,2019-06-30,Very High,4,N/A
198,SCX,STARRETT (L.S.) CO -CL A,2019-06-30,Very High,7,N/A
221,UFI,UNIFI INC,2019-06-30,Very High,5,N/A
269,VAL,VALARIS PLC,2011-12-31,Very High,5,N/A
270,VAL,VALARIS PLC,2012-12-31,Very High,6,N/A
271,VAL,VALARIS PLC,2013-12-31,Very High,6,N/A
272,VAL,VALARIS PLC,2014-12-31,Very High,4,N/A
273,VAL,VALARIS PLC,2015-12-31,Very High,5,N/A


In [6]:
# investigate None purchase date observations in stock_price_df
stock_price_df.loc[stock_price_df['Purchase_Date'].isnull()]

,Ticker_Symbol,Company_Name,Data_Date,BM_Quintile,F_SCORE,Purchase_Date
1362,GMS,GMS INC,2019-04-30,Very High,4,None
1403,BNED,BARNES & NOBLE EDUCATION INC,2019-04-30,Very High,6,None


In [7]:
# note: the purchase date for the 'N/A' observations can be obtained as the SEC filing date
#       for the observations corresponding annual filing
# note: these are investigated and corrected (if possible) below

# observation at index 4 in stock_price_df
stock_price_df.loc[4, 'Purchase_Date'] = '2019-07-18'
# observation at index 44 in stock_price_df
stock_price_df.loc[44, 'Purchase_Date'] = '2019-08-27'
# observation at index 132 in stock_price_df
stock_price_df.loc[132, 'Purchase_Date'] = '2019-08-28'
# observation at index 198 in stock_price_df
stock_price_df.loc[198, 'Purchase_Date'] = '2019-08-26'
# observation at index 221 in stock_price_df
stock_price_df.loc[221, 'Purchase_Date'] = '2019-08-29'
# observation at index 269 in stock_price_df
stock_price_df.loc[269, 'Purchase_Date'] = '2012-02-24'
# observation at index 270 in stock_price_df
stock_price_df.loc[270, 'Purchase_Date'] = '2013-02-22'
# observation at index 271 in stock_price_df
stock_price_df.loc[271, 'Purchase_Date'] = '2014-02-26'
# observation at index 272 in stock_price_df
stock_price_df.loc[272, 'Purchase_Date'] = '2015-03-02'
# observation at index 273 in stock_price_df
stock_price_df.loc[273, 'Purchase_Date'] = '2016-02-25'
# observation at index 274 in stock_price_df
stock_price_df.loc[274, 'Purchase_Date'] = '2017-02-28'
# observation at index 275 in stock_price_df
stock_price_df.loc[275, 'Purchase_Date'] = '2018-02-27'
# observation at index 276 in stock_price_df
stock_price_df.loc[276, 'Purchase_Date'] = '2019-02-28'
# observation at index 1249 in stock_price_df
stock_price_df.loc[1249, 'Purchase_Date'] = '2019-08-14'
# observation at index 1362 in stock_price_df
stock_price_df.loc[1362, 'Purchase_Date'] = '2019-06-27'
# observation at index 1403 in stock_price_df
stock_price_df.loc[1403, 'Purchase_Date'] = '2019-06-25'

In [8]:
# filter out 'N/A' observations
stock_price_df = stock_price_df.query('Purchase_Date!="N/A"')

In [9]:
# check if purchase day is a working day
# if Purchase_Date is not working day, add 1 working day

stock_price_df['Purchase_Date'] = pd.to_datetime(stock_price_df['Purchase_Date'])

for idx, row in stock_price_df.iterrows():
    if cal.is_working_day(row['Purchase_Date'])==True:
        continue
    else:
        stock_price_df.loc[idx, 'Purchase_Date'] = cal.add_working_days(row['Purchase_Date'], 1)

In [10]:
# create column in stock_price_df to store sell date
stock_price_df['Sell_Date'] = 0

# note: sell date will be equal to the Purchase_Date variable plus 1 year
# note: similar to purchase date, if sell date correponds to a holiday, then set sell date
#       to closest date corresponding to business day
for idx, row in stock_price_df.iterrows():
    if datetime.strptime(row['Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') >= datetime(2018, 9, 1):
        stock_price_df.loc[idx, 'Sell_Date'] = datetime(2019, 9, 2).strftime('%Y-%m-%d')
    else:
        if isleap((datetime.strptime(row['Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=365)).year)==True:
            stock_price_df.loc[idx, 'Sell_Date'] = \
            (datetime.strptime(row['Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + 
                                                    timedelta(days=366)).strftime('%Y-%m-%d')
        else:
            stock_price_df.loc[idx, 'Sell_Date'] = \
            (datetime.strptime(row['Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + 
                                                    timedelta(days=365)).strftime('%Y-%m-%d')

In [11]:
# check if Sell_Date in stock_price_df is business day; if not change to closest future business day
for idx, row in stock_price_df.iterrows():
    if cal.is_working_day(datetime.strptime(row['Sell_Date'], '%Y-%m-%d'))==False:
        stock_price_df.loc[idx, 'Sell_Date'] = \
        cal.add_working_days(datetime.strptime(row['Sell_Date'], '%Y-%m-%d'), 1).strftime('%Y-%m-%d')
    else:
        continue

In [12]:
# create column in stock_price_df to store stock purchase price
stock_price_df['Stock_Purchase_Price'] = 0
# create column in stock_price_df to store stock sell price
stock_price_df['Stock_Sell_Price'] = 0

# retrieve Stock_Purchase_Price and Stock_Sell_Price from compustat database
# note: the stock prices are retrieved from the secutiy daily dataset
for idx, row in stock_price_df.iterrows():
    ticker_symbol = row['Ticker_Symbol']
    
    # Stock_Purchase_Price Da
    purchase_date = datetime.strptime(row['Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%m/%d/%Y')
    purchase_price = db.raw_sql("""SELECT DISTINCT prccd
                                   FROM comp.secd WHERE tic=%s
                                   AND datadate=%s""", params=(ticker_symbol, purchase_date))
    # check if purchase_price dataframe is empty
    if purchase_price.empty==True:
        try:
            # if purchase price returns empty dataframe, retrieve data from yahoo finance
            stock_price_df.loc[idx, 'Stock_Purchase_Price'] = \
            data.DataReader(ticker_symbol, 'yahoo', purchase_date, purchase_date)['Close'][0]
        except:
            # if data cannot be obtained, set Stock_Purchase_Price to 'N/A'
            stock_price_df.loc[idx, 'Stock_Purchase_Price'] = 'N/A'
    else:
        stock_price_df.loc[idx, 'Stock_Purchase_Price'] = purchase_price['prccd'][0]
    
    # provide user feedback
    print('-----------------------------------------------------------------------')
    print(idx, ticker_symbol)
    print('stock purchase price: $', stock_price_df.loc[idx, 'Stock_Purchase_Price'], sep='')
    
    # Stock_Sell_Price
    sell_date = datetime.strptime(row['Sell_Date'], '%Y-%m-%d').strftime('%m/%d/%Y')
    sell_price = db.raw_sql("""SELECT DISTINCT prccd
                               FROM comp.secd WHERE tic=%s
                               AND datadate=%s""", params=(ticker_symbol, sell_date))
    # check if sell_price dataframe is empty
    if sell_price.empty==True:
        # check if Stock_Purchase_Price was obtained; if it was obtained proceed to obtain Stock_Sell_Price
        try:
            # if sell price returns empty dataframe, retrieve data from yahoo finance
            stock_price_df.loc[idx, 'Stock_Sell_Price'] = \
            data.DataReader(ticker_symbol, 'yahoo', sell_date, sell_date)['Close'][0]
        except:
            # if data cannot be obtained, set Stock_Sell_Price to 'N/A'
            stock_price_df.loc[idx, 'Stock_Sell_Price'] = 'N/A' 
    else:
        stock_price_df.loc[idx, 'Stock_Sell_Price'] = sell_price['prccd'][0]
    
    print('stock sell price: $', stock_price_df.loc[idx, 'Stock_Sell_Price'], sep='')

-----------------------------------------------------------------------
0 AIR
stock purchase price: $13.63
stock sell price: $25.22
-----------------------------------------------------------------------
1 AIR
stock purchase price: $24.67
stock sell price: $26.85
-----------------------------------------------------------------------
2 AIR
stock purchase price: $27.99
stock sell price: $29.32
-----------------------------------------------------------------------
3 AIR
stock purchase price: $24.26
stock sell price: $36.31
-----------------------------------------------------------------------
4 AIR
stock purchase price: $41.87
stock sell price: $43.13
-----------------------------------------------------------------------
5 HES
stock purchase price: $65.86
stock sell price: $65.68
-----------------------------------------------------------------------
6 HES
stock purchase price: $68.84
stock sell price: $80.35
-----------------------------------------------------------------------
7 HE

-----------------------------------------------------------------------
63 CTL
stock purchase price: $40.25
stock sell price: $34.67
-----------------------------------------------------------------------
64 CTL
stock purchase price: $31.11
stock sell price: $37.68
-----------------------------------------------------------------------
65 CTL
stock purchase price: $30.59
stock sell price: $24.26
-----------------------------------------------------------------------
66 CTL
stock purchase price: $24.26
stock sell price: $17.67
-----------------------------------------------------------------------
67 CTL
stock purchase price: $18.06
stock sell price: $11.56
-----------------------------------------------------------------------
68 CTL
stock purchase price: $12.1
stock sell price: $11.52
-----------------------------------------------------------------------
69 CSS
stock purchase price: $18.32
stock sell price: $20.15
----------------------------------------------------------------------

-----------------------------------------------------------------------
125 GFF
stock purchase price: $12.98
stock sell price: $12.62
-----------------------------------------------------------------------
126 GFF
stock purchase price: $12.67
stock sell price: $17.71
-----------------------------------------------------------------------
127 AXE
stock purchase price: $56.36
stock sell price: $58.63
-----------------------------------------------------------------------
128 JEC
stock purchase price: $43.97
stock sell price: $62.17
-----------------------------------------------------------------------
129 KFY
stock purchase price: $18.71
stock sell price: $30.21
-----------------------------------------------------------------------
130 LCI
stock purchase price: $4.81
stock sell price: $18.09
-----------------------------------------------------------------------
131 LCI
stock purchase price: $5.35
stock sell price: $10.3
-----------------------------------------------------------------

-----------------------------------------------------------------------
187 LUV
stock purchase price: $9.62
stock sell price: $11.65
-----------------------------------------------------------------------
188 LUV
stock purchase price: $11.64
stock sell price: $21.02
-----------------------------------------------------------------------
189 CVX
stock purchase price: $83.44
stock sell price: $112.5
-----------------------------------------------------------------------
190 SCX
stock purchase price: $10.44
stock sell price: $12.59
-----------------------------------------------------------------------
191 SCX
stock purchase price: $12.7
stock sell price: $11.0
-----------------------------------------------------------------------
192 SCX
stock purchase price: $10.96
stock sell price: $15.99
-----------------------------------------------------------------------
193 SCX
stock purchase price: $15.19
stock sell price: $13.0
------------------------------------------------------------------

247 GHC
stock purchase price: $395.83
stock sell price: $701.49
-----------------------------------------------------------------------
248 GHC
stock purchase price: $499.27
stock sell price: $568.25
-----------------------------------------------------------------------
249 GHC
stock purchase price: $571.15
stock sell price: $589.1
-----------------------------------------------------------------------
250 GHC
stock purchase price: $584.85
stock sell price: $674.34
-----------------------------------------------------------------------
251 GHC
stock purchase price: $678.19
stock sell price: $701.15
-----------------------------------------------------------------------
252 AVA
stock purchase price: $25.25
stock sell price: $26.46
-----------------------------------------------------------------------
253 AVA
stock purchase price: $26.17
stock sell price: $30.0
-----------------------------------------------------------------------
254 WMK
stock purchase price: $38.31
stock sell price:

...

stock sell price: $26.35
-----------------------------------------------------------------------
1180 TOWR
stock purchase price: $14.05
stock sell price: $25.35
-----------------------------------------------------------------------
1181 OAS
stock purchase price: $13.93
stock sell price: $7.2
-----------------------------------------------------------------------
1182 OAS
stock purchase price: $7.99
stock sell price: $12.58
-----------------------------------------------------------------------
1183 OAS
stock purchase price: $8.1
stock sell price: $6.04
-----------------------------------------------------------------------
1184 OAS
stock purchase price: $5.95
stock sell price: $3.05
-----------------------------------------------------------------------
1185 VPG
stock purchase price: $14.38
stock sell price: $18.1
-----------------------------------------------------------------------
1186 VPG
stock purchase price: $17.34
stock sell price: $15.53
--------------------------------------

stock sell price: $20.1
-----------------------------------------------------------------------
1242 TROX
stock purchase price: $6.24
stock sell price: $16.5
-----------------------------------------------------------------------
1243 TROX
stock purchase price: $16.66
stock sell price: $18.23
-----------------------------------------------------------------------
1244 TROX
stock purchase price: $11.75
stock sell price: $7.08
-----------------------------------------------------------------------
1245 POST
stock purchase price: $41.02
stock sell price: $67.58
-----------------------------------------------------------------------
1246 EROS
stock purchase price: $17.83
stock sell price: $10.85
-----------------------------------------------------------------------
1247 EROS
stock purchase price: $8.7
stock sell price: $12.95
-----------------------------------------------------------------------
1248 EROS
stock purchase price: $13.1
stock sell price: $1.24
-------------------------------

-----------------------------------------------------------------------
1303 ASC
stock purchase price: $8.0
stock sell price: $6.96
-----------------------------------------------------------------------
1304 ASC
stock purchase price: $5.99
stock sell price: $6.15
-----------------------------------------------------------------------
1305 DLNG
stock purchase price: $13.71
stock sell price: $15.57
-----------------------------------------------------------------------
1306 DLNG
stock purchase price: $17.26
stock sell price: $9.59
-----------------------------------------------------------------------
1307 DLNG
stock purchase price: $7.71
stock sell price: $2.37
-----------------------------------------------------------------------
1308 DLNG
stock purchase price: $1.71
stock sell price: $1.31
-----------------------------------------------------------------------
1309 AGN
stock purchase price: $244.82
stock sell price: $154.22
-----------------------------------------------------------

stock sell price: $3.07
-----------------------------------------------------------------------
1365 VEC
stock purchase price: $26.96
stock sell price: $40.54
-----------------------------------------------------------------------
1366 HMLP
stock purchase price: $18.56
stock sell price: $14.3
-----------------------------------------------------------------------
1367 NEP
stock purchase price: $30.62
stock sell price: $38.76
-----------------------------------------------------------------------
1368 NEP
stock purchase price: $37.46
stock sell price: $45.18
-----------------------------------------------------------------------
1369 NEP
stock purchase price: $44.6
stock sell price: $51.27
-----------------------------------------------------------------------
1370 EURN
stock purchase price: $11.14
stock sell price: $7.95
-----------------------------------------------------------------------
1371 EURN
stock purchase price: $7.8
stock sell price: $8.6
-----------------------------------

stock sell price: $37.7
-----------------------------------------------------------------------
1426 DK
stock purchase price: $15.18
stock sell price: $24.17
-----------------------------------------------------------------------
1427 KL
stock purchase price: $3.09
stock sell price: $3.6586
-----------------------------------------------------------------------
1428 KL
stock purchase price: $3.632
stock sell price: $4.0507
-----------------------------------------------------------------------
1429 HPR
stock purchase price: $19.66
stock sell price: $23.75
-----------------------------------------------------------------------
1430 HPR
stock purchase price: $8.22
stock sell price: $5.4
-----------------------------------------------------------------------
1431 HPR
stock purchase price: $5.68
stock sell price: $4.17
-----------------------------------------------------------------------
1432 HPR
stock purchase price: $4.46
stock sell price: $4.82
----------------------------------------

In [13]:
# investigate observations with 'N/A' Stock_Purchase_Price or Stock_Sell_Price 
stock_price_df.query('Stock_Purchase_Price=="N/A" | Stock_Sell_Price=="N/A"')

,Ticker_Symbol,Company_Name,Data_Date,BM_Quintile,F_SCORE,Purchase_Date,Sell_Date,Stock_Purchase_Price,Stock_Sell_Price
59,CATO,CATO CORP -CL A,2018-01-31,Very High,5,2018-03-30,2019-04-01,N/A,15.28
163,OSG,OVERSEAS SHIPHOLDING GROUP,2011-12-31,Very High,1,2012-03-20,2013-03-20,N/A,N/A
164,OSG,OVERSEAS SHIPHOLDING GROUP,2012-12-31,Very High,4,2013-09-06,2014-09-08,N/A,N/A
171,PKD,PARKER DRILLING CO,2012-12-31,Very High,6,2013-03-25,2014-03-25,N/A,N/A
172,PKD,PARKER DRILLING CO,2014-12-31,Very High,5,2015-03-19,2016-03-21,N/A,N/A
173,PKD,PARKER DRILLING CO,2015-12-31,Very High,3,2016-03-22,2017-03-22,N/A,N/A
174,PKD,PARKER DRILLING CO,2016-12-31,Very High,3,2017-03-17,2018-03-19,N/A,N/A
175,PKD,PARKER DRILLING CO,2017-12-31,Very High,6,2018-04-09,2019-04-09,N/A,17.64
204,SUP,SUPERIOR INDUSTRIES INTL,2017-12-31,Very High,5,2018-03-30,2019-04-01,N/A,5.51
288,SBOW,SILVERBOW RESOURCES INC,2012-12-31,Very High,3,2013-03-06,2014-03-06,N/A,N/A


In [14]:
# manually input 'N/A' observations by looking up on compustat

# observation for index 59
# change Purchase_Date to 2018-04-02
stock_price_df.loc[59, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[59, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') - timedelta(days=3)
stock_price_df.loc[59, 'Stock_Purchase_Price'] = 14.62

# observation for index 163
stock_price_df.loc[163, 'Stock_Purchase_Price'] = 11.29
stock_price_df.loc[163, 'Stock_Sell_Price'] = 2.18

# observation for index 164
stock_price_df.loc[164, 'Stock_Purchase_Price'] = 3.07
stock_price_df.loc[164, 'Stock_Sell_Price'] = 6.05

# observation for index 171
stock_price_df.loc[171, 'Stock_Purchase_Price'] = 4.62
stock_price_df.loc[171, 'Stock_Sell_Price'] = 7.24

# observation for index 172
stock_price_df.loc[172, 'Stock_Purchase_Price'] = 3.11
stock_price_df.loc[172, 'Stock_Sell_Price'] = 2.28

# observation for index 173
stock_price_df.loc[173, 'Stock_Purchase_Price'] = 2.14
stock_price_df.loc[173, 'Stock_Sell_Price'] = 1.45

# observation for index 174
stock_price_df.loc[174, 'Stock_Purchase_Price'] = 1.6
stock_price_df.loc[174, 'Stock_Sell_Price'] = 0.7619

# observation for index 175
stock_price_df.loc[175, 'Stock_Purchase_Price'] = 0.493

# observation for index 204
# change Purchase_Date to 2018-04-02
stock_price_df.loc[204, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[204, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[204, 'Stock_Purchase_Price'] = 13.05

# observation for index 288
stock_price_df.loc[288, 'Stock_Purchase_Price'] = 13.9
stock_price_df.loc[288, 'Stock_Sell_Price'] = 9.81

# observation for index 289
stock_price_df.loc[289, 'Stock_Purchase_Price'] = 10.4
stock_price_df.loc[289, 'Stock_Sell_Price'] = 2.18

# observation for index 290
stock_price_df.loc[290, 'Stock_Purchase_Price'] = 2.18
stock_price_df.loc[290, 'Stock_Sell_Price'] = 0.09

# observation for index 500
# change Purchase_Date to 2018-04-02
stock_price_df.loc[500, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[500, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[500, 'Stock_Sell_Price'] = 20.88

# observation for index 503
# change Purchase_Date to 2019-04-22
stock_price_df.loc[503, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[503, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[503, 'Stock_Purchase_Price'] = 8.14

# observation for index 523
stock_price_df.loc[523, 'Stock_Purchase_Price'] = 7.01
stock_price_df.loc[523, 'Stock_Sell_Price'] = 10.26

# observation for index 524
stock_price_df.loc[524, 'Stock_Purchase_Price'] = 10.16
stock_price_df.loc[524, 'Stock_Sell_Price'] = 4.02

# observation for index 525
# cannot source pricing data; therefore leave as 'N/A'

# observation for index 555
# change Purchase_Date to 2015-04-06
stock_price_df.loc[555, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[555, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[555, 'Stock_Purchase_Price'] = 9.38

# observation for index 646
stock_price_df.loc[646, 'Stock_Purchase_Price'] = 12.29
stock_price_df.loc[646, 'Stock_Sell_Price'] = 5.46

# observation for index 647
stock_price_df.loc[647, 'Stock_Purchase_Price'] = 5.46
stock_price_df.loc[647, 'Stock_Sell_Price'] = 4.35

# observation for index 648
stock_price_df.loc[648, 'Stock_Purchase_Price'] = 4.69
stock_price_df.loc[648, 'Stock_Sell_Price'] = 1.03

# observation for index 649
stock_price_df.loc[649, 'Stock_Purchase_Price'] = 0.9048

# observation for index 684
stock_price_df.loc[684, 'Stock_Purchase_Price'] = 6.35
stock_price_df.loc[684, 'Stock_Sell_Price'] = 5.89

# observation for index 685
stock_price_df.loc[685, 'Stock_Purchase_Price'] = 2.41
stock_price_df.loc[685, 'Stock_Sell_Price'] = 2.22

# observation for index 762
# change Purchase_Date to 2019-04-22
stock_price_df.loc[762, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[762, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[762, 'Stock_Sell_Price'] = 1.48

# observation for index 797
# change Purchase_Date to 2018-04-02
stock_price_df.loc[797, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[797, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[797, 'Stock_Purchase_Price'] = 2.99

# observation for index 921
stock_price_df.loc[921, 'Stock_Purchase_Price'] = 6.87
stock_price_df.loc[921, 'Stock_Sell_Price'] = 2.79

# observation for index 922
stock_price_df.loc[922, 'Stock_Purchase_Price'] = 3.01
stock_price_df.loc[922, 'Stock_Sell_Price'] = 1.54

# observation for index 923
stock_price_df.loc[923, 'Stock_Purchase_Price'] = 1.7

# observation for index 930
# change Purchase_Date to 2013-04-01
stock_price_df.loc[930, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[930, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[930, 'Stock_Sell_Price'] = 29.76

# observation for index 932
# change Purchase_Date to 2018-04-02
stock_price_df.loc[932, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[932, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[932, 'Stock_Sell_Price'] = 12.82

# observation for index 937
# change Purchase_Date to 2015-04-06
stock_price_df.loc[937, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[937, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[937, 'Stock_Sell_Price'] = 7.24

# observation for index 966
# change Purchase_Date to 2018-04-30
stock_price_df.loc[966, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[966, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[966, 'Stock_Sell_Price'] = 1.04

# observation for index 976
stock_price_df.loc[976, 'Stock_Purchase_Price'] = 5.38
stock_price_df.loc[976, 'Stock_Sell_Price'] = 6.38

# observation for index 977
stock_price_df.loc[977, 'Stock_Purchase_Price'] = 6.38
stock_price_df.loc[977, 'Stock_Sell_Price'] = 1.57

# observation for index 978
stock_price_df.loc[978, 'Stock_Purchase_Price'] = 1.77
stock_price_df.loc[978, 'Stock_Sell_Price'] = 0.0426

# observation for index 995
# change Purchase_Date to 2013-04-01
stock_price_df.loc[995, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[995, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[995, 'Stock_Purchase_Price'] = 4.11

# observation for index 996
# change Purchase_Date to 2015-04-06
stock_price_df.loc[996, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[996, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[996, 'Stock_Sell_Price'] = 6.46

# observation for index 1072
# change Purchase_Date to 2017-04-17
stock_price_df.loc[1072, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1072, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1072, 'Stock_Purchase_Price'] = 13.9

# observation for index 1240
# change Purchase_Date to 2018-04-02
stock_price_df.loc[1240, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[1240, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1240, 'Stock_Sell_Price'] = 89.44

# observation for index 1258
# change Purchase_Date to 2017-04-17
stock_price_df.loc[1258, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1258, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1258, 'Stock_Purchase_Price'] = 0.128

# observation for index 1271
# change Purchase_Date to 2019-04-22
stock_price_df.loc[1271, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1271, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1271, 'Stock_Purchase_Price'] = 1.56

# observation for index 1332
# change Purchase_Date to 2019-04-22
stock_price_df.loc[1332, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1332, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1332, 'Stock_Purchase_Price'] = 2.12

# observation for index 1376
# change Purchase_Date to 2019-04-22
stock_price_df.loc[1376, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1376, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1376, 'Stock_Purchase_Price'] = 22.67

# observation for index 1384
stock_price_df.loc[1384, 'Stock_Purchase_Price'] = 11.9
stock_price_df.loc[1384, 'Stock_Sell_Price'] = 1.75

# observation for index 1385
# change Purchase_Date to 2019-03-28
stock_price_df.loc[1385, 'Purchase_Date'] = datetime.strptime(stock_price_df.loc[1385, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1385, 'Stock_Purchase_Price'] = 1.46

# observation for index 1390
# cannot source pricing data; therefore leave as 'N/A'

# observation for index 1397
# change Purchase_Date to 2018-04-02
stock_price_df.loc[1397, 'Sell_Date'] = datetime.strptime(stock_price_df.loc[1397, 'Sell_Date'], '%Y-%m-%d') + timedelta(days=3)
stock_price_df.loc[1397, 'Stock_Sell_Price'] = 9.84

In [15]:
stock_price_df.count()

Ticker_Symbol           1455
Company_Name            1455
Data_Date               1455
BM_Quintile             1455
F_SCORE                 1455
Purchase_Date           1455
Sell_Date               1455
Stock_Purchase_Price    1455
Stock_Sell_Price        1455
dtype: int64

## Calculation of Holding Period Returns (Long and Short)

In [16]:
# calculate holding period return for a unit of stock (i.e. one stock in each observation in stock_price_df)
# the return for a long position and short position will be calculated
# note: the return be stored in a new column in stock_price_df

# create long holding period return column in stock_price_df
stock_price_df['Long_Holding_Period_Return_(%)'] = 0
# create short holding period return column in stock_price_df
stock_price_df['Short_Holding_Period_Return_(%)'] = 0
# calculate Long_Holding_Period_Return and Short_Holding_Period_Return_(%)
# note: long holding period return return = (stock sell price / stock purchase price) - 1
# note: short holding period return = (stock purchase price / stock sell price) - 1
# check if Stock_Purchase_Price and Stock_Sell_Price are not equal to 'N/A'
# if Stock_Purchase_Price and Stock_Sell_Price are equal to 'N/A', set long holding period return and short holding period return to 'N/A'
# otherwise long holding period return and short holding period return equals that of the prvious formulas above
for idx, row in stock_price_df.iterrows():
    if (row['Stock_Purchase_Price']=='N/A') & (row['Stock_Sell_Price']=='N/A'):
        stock_price_df.loc[idx, 'Long_Holding_Period_Return_(%)'] = 'N/A'
        stock_price_df.loc[idx, 'Short_Holding_Period_Return_(%)'] = 'N/A'
    else:
        stock_price_df.loc[idx, 'Long_Holding_Period_Return_(%)'] = (row['Stock_Sell_Price'] / row['Stock_Purchase_Price']) - 1
        stock_price_df.loc[idx, 'Short_Holding_Period_Return_(%)'] = (row['Stock_Purchase_Price'] / row['Stock_Sell_Price']) - 1

In [17]:
# convert 'Sell_Date' column to timestamp in stock_price_df
stock_price_df['Sell_Date'] = pd.to_datetime(stock_price_df['Sell_Date'])

## Calculation of Annualized Holding Period Returns

In [18]:
# note: to compare against a suitable benchmark, it is necessary to calculate the annualized hold period return of each
#       observation in stock_price_df
#       the annualized holding period return is given by (1 + holding period return)^(365/days stock has been traded) - 1

# create new column to store long annualized holding period returns
stock_price_df['Long_Annualized_Holding_Period_Return_(%)'] = ''
# create new column to store short annualized holding period returns
stock_price_df['Short_Annualized_Holding_Period_Return_(%)'] = ''

for idx, row in stock_price_df.iterrows():
    if row['Long_Holding_Period_Return_(%)']=='N/A' and row['Short_Holding_Period_Return_(%)']=='N/A':
        stock_price_df.loc[idx, 'Long_Annualized_Holding_Period_Return_(%)'] = 'N/A'
        stock_price_df.loc[idx, 'Short_Annualized_Holding_Period_Return_(%)'] = 'N/A'
    else:
        purchase_date = datetime.strptime(stock_price_df.loc[idx, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d')
        sell_date = datetime.strptime(stock_price_df.loc[idx, 'Sell_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d')
        holding_days = (sell_date - purchase_date).days
        stock_price_df.loc[idx, 'Long_Annualized_Holding_Period_Return_(%)'] = ((1 + row['Long_Holding_Period_Return_(%)'])**(365/holding_days) - 1)
        stock_price_df.loc[idx, 'Short_Annualized_Holding_Period_Return_(%)'] = ((1 + row['Short_Holding_Period_Return_(%)'])**(365/holding_days) - 1)

## COMPUSTAT - SPDR S&P 500 ETF Data Extraction

In [19]:
# note: SPDR S&P 500 is a ETF which 'mimics' the movements of the S&P 500 index
# note: the return of the S&P 500 is used throughout investment management to assess the performance of portfolios
# note: the index acts as a benchmark and as a component to calculating excess returns
# note: the long holding period return and short holding period return will be calculated for SPDR relative to each observation in stock_price_df

# create a new column to store the purchase price of SPDR
stock_price_df['SPDR_Purchase_Price'] = ''
# create a new column to store the sell price of SPDR
stock_price_df['SPDR_Sell_Price'] = ''

# retrive the purchase price and sell price of the SPDR ETF for the purchase date and sell date for each observation in stock_price_df
for idx, row in stock_price_df.iterrows():
    if row['Long_Holding_Period_Return_(%)']=='N/A' and row['Short_Holding_Period_Return_(%)']=='N/A':
        stock_price_df.loc[idx, 'SPDR_Purchase_Price'] = 'N/A'
        stock_price_df.loc[idx, 'SPDR_Sell_Price'] = 'N/A'
    else:
        spdr_purchase_price = db.raw_sql("""SELECT DISTINCT prccd
                                            FROM comp.secd WHERE tic='SPY'
                                            AND datadate=%s""", params=([row['Purchase_Date'].strftime('%Y-%m-%d')]))
        spdr_sell_price = db.raw_sql("""SELECT DISTINCT prccd
                                            FROM comp.secd WHERE tic='SPY'
                                            AND datadate=%s""", params=([row['Sell_Date'].strftime('%Y-%m-%d')]))
        stock_price_df.loc[idx, 'SPDR_Purchase_Price'] = spdr_purchase_price['prccd'][0]
        stock_price_df.loc[idx, 'SPDR_Sell_Price'] = spdr_sell_price['prccd'][0]
        # provide user feedback
        print('-----------------------------------------------------------------------')
        print(idx, row['Ticker_Symbol'])
        print('spdr purchase date:', row['Purchase_Date'].strftime('%Y-%m-%d'))
        print('spdr purchase price $', spdr_purchase_price['prccd'][0], sep='')
        print('spdr sell date', row['Sell_Date'].strftime('%Y-%m-%d'))
        print('spdr sell price $', spdr_sell_price['prccd'][0], sep='')

-----------------------------------------------------------------------
0 AIR
spdr purchase date: 2012-07-23
spdr purchase price $135.09
spdr sell date 2013-07-23
spdr sell price $169.14
-----------------------------------------------------------------------
1 AIR
spdr purchase date: 2013-08-01
spdr purchase price $170.66
spdr sell date 2014-08-01
spdr sell price $192.5
-----------------------------------------------------------------------
2 AIR
spdr purchase date: 2014-07-21
spdr purchase price $197.34
spdr sell date 2015-07-21
spdr sell price $211.75
-----------------------------------------------------------------------
3 AIR
spdr purchase date: 2016-07-18
spdr purchase price $216.4092
spdr sell date 2017-07-18
spdr sell price $245.66
-----------------------------------------------------------------------
4 AIR
spdr purchase date: 2019-07-18
spdr purchase price $298.83
spdr sell date 2019-09-03
spdr sell price $290.74
----------------------------------------------------------------

-----------------------------------------------------------------------
44 BGG
spdr purchase date: 2019-08-27
spdr purchase price $286.87
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
45 CAL
spdr purchase date: 2012-04-04
spdr purchase price $139.86
spdr sell date 2013-04-04
spdr sell price $155.86
-----------------------------------------------------------------------
46 CBT
spdr purchase date: 2011-12-08
spdr purchase price $123.95
spdr sell date 2012-12-10
spdr sell price $142.473
-----------------------------------------------------------------------
47 CBT
spdr purchase date: 2012-12-03
spdr purchase price $141.45
spdr sell date 2013-12-03
spdr sell price $179.75
-----------------------------------------------------------------------
48 CACI
spdr purchase date: 2012-09-06
spdr purchase price $143.77
spdr sell date 2013-09-06
spdr sell price $166.04
----------------------------------------------------------

spdr sell date 2013-03-08
spdr sell price $155.44
-----------------------------------------------------------------------
89 CW
spdr purchase date: 2013-02-27
spdr purchase price $151.91
spdr sell date 2014-02-27
spdr sell price $185.82
-----------------------------------------------------------------------
90 DAN
spdr purchase date: 2012-03-07
spdr purchase price $135.69
spdr sell date 2013-03-07
spdr sell price $154.776
-----------------------------------------------------------------------
91 DAN
spdr purchase date: 2013-02-26
spdr purchase price $150.02
spdr sell date 2014-02-26
spdr sell price $184.85
-----------------------------------------------------------------------
92 DDS
spdr purchase date: 2011-04-01
spdr purchase price $133.15
spdr sell date 2012-04-02
spdr sell price $141.84
-----------------------------------------------------------------------
93 DDS
spdr purchase date: 2012-04-03
spdr purchase price $141.26
spdr sell date 2013-04-03
spdr sell price $155.23
----------

-----------------------------------------------------------------------
133 VHI
spdr purchase date: 2016-03-24
spdr purchase price $203.12
spdr sell date 2017-03-24
spdr sell price $233.86
-----------------------------------------------------------------------
134 VHI
spdr purchase date: 2019-03-28
spdr purchase price $280.71
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
135 LPX
spdr purchase date: 2012-03-19
spdr purchase price $140.85
spdr sell date 2013-03-19
spdr sell price $154.61
-----------------------------------------------------------------------
136 LXU
spdr purchase date: 2016-03-22
spdr purchase price $204.56
spdr sell date 2017-03-22
spdr sell price $234.28
-----------------------------------------------------------------------
137 LXU
spdr purchase date: 2017-03-15
spdr purchase price $238.95
spdr sell date 2018-03-15
spdr sell price $275.0
--------------------------------------------------------

-----------------------------------------------------------------------
178 TPC
spdr purchase date: 2013-03-20
spdr purchase price $155.69
spdr sell date 2014-03-20
spdr sell price $187.747
-----------------------------------------------------------------------
179 TPC
spdr purchase date: 2014-03-10
spdr purchase price $188.16
spdr sell date 2015-03-10
spdr sell price $204.98
-----------------------------------------------------------------------
180 TPC
spdr purchase date: 2015-03-16
spdr purchase price $208.58
spdr sell date 2016-03-16
spdr sell price $203.34
-----------------------------------------------------------------------
181 TPC
spdr purchase date: 2016-03-15
spdr purchase price $202.17
spdr sell date 2017-03-15
spdr sell price $238.95
-----------------------------------------------------------------------
182 TPC
spdr purchase date: 2017-03-09
spdr purchase price $236.86
spdr sell date 2018-03-09
spdr sell price $278.87
------------------------------------------------------

...

spdr sell date 2018-04-02
spdr sell price $257.47
-----------------------------------------------------------------------
1241 TROX
spdr purchase date: 2014-03-17
spdr purchase price $186.33
spdr sell date 2015-03-17
spdr sell price $207.96
-----------------------------------------------------------------------
1242 TROX
spdr purchase date: 2016-03-22
spdr purchase price $204.56
spdr sell date 2017-03-22
spdr sell price $234.28
-----------------------------------------------------------------------
1243 TROX
spdr purchase date: 2017-03-23
spdr purchase price $234.03
spdr sell date 2018-03-23
spdr sell price $258.05
-----------------------------------------------------------------------
1244 TROX
spdr purchase date: 2019-03-14
spdr purchase price $281.16
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
1245 POST
spdr purchase date: 2014-12-04
spdr purchase price $207.66
spdr sell date 2015-12-04
spdr sell price $20

-----------------------------------------------------------------------
1284 MPLX
spdr purchase date: 2017-03-21
spdr purchase price $233.73
spdr sell date 2018-03-21
spdr sell price $270.43
-----------------------------------------------------------------------
1285 SUN
spdr purchase date: 2016-03-17
spdr purchase price $204.63
spdr sell date 2017-03-17
spdr sell price $237.03
-----------------------------------------------------------------------
1286 SUN
spdr purchase date: 2017-03-20
spdr purchase price $236.77
spdr sell date 2018-03-20
spdr sell price $270.95
-----------------------------------------------------------------------
1287 ANFI
spdr purchase date: 2016-01-28
spdr purchase price $189.11
spdr sell date 2017-01-27
spdr sell price $228.97
-----------------------------------------------------------------------
1288 ANFI
spdr purchase date: 2016-08-19
spdr purchase price $218.54
spdr sell date 2017-08-21
spdr sell price $242.9
------------------------------------------------

-----------------------------------------------------------------------
1327 SALT
spdr purchase date: 2019-04-12
spdr purchase price $290.16
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
1328 KN
spdr purchase date: 2018-03-09
spdr purchase price $278.87
spdr sell date 2019-03-11
spdr sell price $278.44
-----------------------------------------------------------------------
1329 KN
spdr purchase date: 2019-03-01
spdr purchase price $280.42
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
1330 CVEO
spdr purchase date: 2017-04-06
spdr purchase price $235.44
spdr sell date 2018-04-06
spdr sell price $259.72
-----------------------------------------------------------------------
1331 CVEO
spdr purchase date: 2018-04-20
spdr purchase price $266.61
spdr sell date 2019-04-22
spdr sell price $290.27
-------------------------------------------------

-----------------------------------------------------------------------
1371 EURN
spdr purchase date: 2017-04-13
spdr purchase price $232.51
spdr sell date 2018-04-13
spdr sell price $265.15
-----------------------------------------------------------------------
1372 EURN
spdr purchase date: 2018-05-23
spdr purchase price $273.36
spdr sell date 2019-05-23
spdr sell price $282.14
-----------------------------------------------------------------------
1373 EURN
spdr purchase date: 2019-04-30
spdr purchase price $294.02
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
1374 WLKP
spdr purchase date: 2017-04-13
spdr purchase price $232.51
spdr sell date 2018-04-13
spdr sell price $265.15
-----------------------------------------------------------------------
1375 WLKP
spdr purchase date: 2018-04-24
spdr purchase price $262.98
spdr sell date 2019-04-24
spdr sell price $292.23
---------------------------------------------

-----------------------------------------------------------------------
1416 FTS
spdr purchase date: 2017-02-22
spdr purchase price $236.28
spdr sell date 2018-02-22
spdr sell price $270.4
-----------------------------------------------------------------------
1417 FTS
spdr purchase date: 2018-02-26
spdr purchase price $277.9
spdr sell date 2019-02-26
spdr sell price $279.32
-----------------------------------------------------------------------
1418 FTS
spdr purchase date: 2019-03-15
spdr purchase price $281.31
spdr sell date 2019-09-03
spdr sell price $290.74
-----------------------------------------------------------------------
1419 ASH
spdr purchase date: 2011-12-08
spdr purchase price $123.95
spdr sell date 2012-12-10
spdr sell price $142.473
-----------------------------------------------------------------------
1420 ASH
spdr purchase date: 2017-11-29
spdr purchase price $262.71
spdr sell date 2018-11-29
spdr sell price $273.98
---------------------------------------------------

## Calculation of SPDR S&P 500 ETF Holding Period Return (Long and Short)

In [20]:
# calculate holding period return for a unit of SPDR S&P 500 ETF (i.e. one stock of the ETF)
# the return for a long position and short position will be calculated
# note: the return be stored in a new column in stock_price_df

# create long holding period return column in stock_price_df
stock_price_df['SPDR_Long_Holding_Period_Return_(%)'] = 0
# create short holding period return column in stock_price_df
stock_price_df['SPDR_Short_Holding_Period_Return_(%)'] = 0
# calculate SPDR_Long_Holding_Period_Return_(%) and SPDR_Short_Holding_Period_Return_(%)
# note: long holding period return return = (stock sell price / stock purchase price) - 1
# note: short holding period return = (stock purchase price / stock sell price) - 1
# check if SPDR_Purchase_Price and SPDR_Sell_Price are not equal to 'N/A'
# if SPDR_Purchase_Price and SPDR_Sell_Price are equal to 'N/A', set long holding period return and short holding period return to 'N/A'
# otherwise long holding period return and short holding period return equals that of the prvious formulas above
for idx, row in stock_price_df.iterrows():
    if (row['SPDR_Purchase_Price']=='N/A') & (row['SPDR_Sell_Price']=='N/A'):
        stock_price_df.loc[idx, 'SPDR_Long_Holding_Period_Return_(%)'] = 'N/A'
        stock_price_df.loc[idx, 'SPDR_Short_Holding_Period_Return_(%)'] = 'N/A'
    else:
        stock_price_df.loc[idx, 'SPDR_Long_Holding_Period_Return_(%)'] = (row['SPDR_Sell_Price'] / row['SPDR_Purchase_Price']) - 1
        stock_price_df.loc[idx, 'SPDR_Short_Holding_Period_Return_(%)'] = (row['SPDR_Purchase_Price'] / row['SPDR_Sell_Price']) - 1

## Calculation of SPDR S&P 500 ETF Annualized Holding Period Returns

In [21]:
# create new column to store spdr long annualized holding period returns
stock_price_df['SPDR_Long_Annualized_Holding_Period_Return_(%)'] = ''
# create new column to store spdr short annualized holding period returns
stock_price_df['SPDR_Short_Annualized_Holding_Period_Return_(%)'] = ''

for idx, row in stock_price_df.iterrows():
    if row['SPDR_Long_Holding_Period_Return_(%)']=='N/A' and row['SPDR_Short_Holding_Period_Return_(%)']=='N/A':
        stock_price_df.loc[idx, 'SPDR_Long_Annualized_Holding_Period_Return_(%)'] = 'N/A'
        stock_price_df.loc[idx, 'SPDR_Short_Annualized_Holding_Period_Return_(%)'] = 'N/A'
    else:
        purchase_date = datetime.strptime(stock_price_df.loc[idx, 'Purchase_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d')
        sell_date = datetime.strptime(stock_price_df.loc[idx, 'Sell_Date'].strftime('%Y-%m-%d'), '%Y-%m-%d')
        holding_days = (sell_date - purchase_date).days
        stock_price_df.loc[idx, 'SPDR_Long_Annualized_Holding_Period_Return_(%)'] = ((1 + row['SPDR_Long_Holding_Period_Return_(%)'])**(365/holding_days) - 1)
        stock_price_df.loc[idx, 'SPDR_Short_Annualized_Holding_Period_Return_(%)'] = ((1 + row['SPDR_Short_Holding_Period_Return_(%)'])**(365/holding_days) - 1)

## Calculation of Excess Returns (Long and Short)

In [22]:
# note: excess return is a measure against the performance of stocks and a given benchmark (i.e. the spdr s&p 500 etf)
# note: in simple terms excess return = stock return - benchmark return 
# note: the product of excess returns is known as alpha
# note: to calculate excess returns, the annualized returns will be used

# create new column to store long (annualized) excess returns in stock_price_df
stock_price_df['Long_Excess_Returns_(%)'] = 0
# create new column to store short (annualized) excess returns in stock_price_df
stock_price_df['Short_Excess_Returns_(%)'] = 0


for idx, row in stock_price_df.iterrows():
    if row['Stock_Purchase_Price']=='N/A' and row['Stock_Sell_Price']=='N/A':
        stock_price_df.loc[idx, 'Long_Excess_Returns_(%)'] = 'N/A'
        stock_price_df.loc[idx, 'Short_Excess_Returns_(%)'] = 'N/A'
    else:
        stock_price_df.loc[idx, 'Long_Excess_Returns_(%)'] = row['Long_Annualized_Holding_Period_Return_(%)'] - \
                                                                row['SPDR_Long_Annualized_Holding_Period_Return_(%)']
        stock_price_df.loc[idx, 'Short_Excess_Returns_(%)'] = row['Short_Annualized_Holding_Period_Return_(%)'] - \
                                                                row['SPDR_Short_Annualized_Holding_Period_Return_(%)']

## Portfolio Analysis

In [42]:
# note: various portfolio formations will be analyzed to identify the best combination of stocks
# note: a function will be defined to calculate various statistics of the various portfolios

def portfolio_analysis_statistics(portfolio_df):
    # mean of returns
    average_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].mean()
    average_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].mean()
    average_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].mean()
    average_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].mean()
    average_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].mean()
    average_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].mean()
    # median returns
    median_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].median()
    median_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].median()
    median_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].median()
    median_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].median()
    median_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].median()
    median_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].median()
    # 10th percentile returns
    percentile_10_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].quantile(0.1)
    percentile_10_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].quantile(0.1)
    percentile_10_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].quantile(0.1)
    percentile_10_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].quantile(0.1)
    percentile_10_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].quantile(0.1)
    percentile_10_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].quantile(0.1)
    # 25th percentile returns 
    percentile_25_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].quantile(0.25)
    percentile_25_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].quantile(0.25)
    percentile_25_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].quantile(0.25)
    percentile_25_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].quantile(0.25)
    percentile_25_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].quantile(0.25)
    percentile_25_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].quantile(0.25)
    # 75th percentile returns
    percentile_75_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].quantile(0.75)
    percentile_75_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].quantile(0.75)
    percentile_75_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].quantile(0.75)
    percentile_75_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].quantile(0.75)
    percentile_75_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].quantile(0.75)
    percentile_75_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].quantile(0.75)
    # 90th percentile returns
    percentile_90_long_hpr = portfolio_df['Long_Holding_Period_Return_(%)'].quantile(0.9)
    percentile_90_short_hpr = portfolio_df['Short_Holding_Period_Return_(%)'].quantile(0.9)
    percentile_90_long_annaulized_hpr = portfolio_df['Long_Annualized_Holding_Period_Return_(%)'].quantile(0.9)
    percentile_90_short_annaulized_hpr = portfolio_df['Short_Annualized_Holding_Period_Return_(%)'].quantile(0.9)
    percentile_90_long_excess_returns = portfolio_df['Long_Excess_Returns_(%)'].quantile(0.9)
    percentile_90_short_excess_returns = portfolio_df['Short_Excess_Returns_(%)'].quantile(0.9)

    print('number of observations:', portfolio_df.shape[0])
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('mean returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((average_long_hpr), 3))
    print('short holding period return:', round((average_short_hpr), 3))
    print('long annualized holding period return:', round((average_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((average_short_annaulized_hpr), 3))
    print('long excess return:', round((average_long_excess_returns), 3))
    print('short excess return:', round((average_short_excess_returns), 3))
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('median returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((median_long_hpr), 3))
    print('short holding period return:', round((median_short_hpr), 3))
    print('long annualized holding period return:', round((median_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((median_short_annaulized_hpr), 3))
    print('long excess return:', round((median_long_excess_returns), 3))
    print('short excess return:', round((median_short_excess_returns), 3))
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('10th percentile returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((percentile_10_long_hpr), 3))
    print('short holding period return:', round((percentile_10_short_hpr), 3))
    print('long annualized holding period return:', round((percentile_10_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((percentile_10_short_annaulized_hpr), 3))
    print('long excess return:', round((percentile_10_long_excess_returns), 3))
    print('short excess return:', round((percentile_10_short_excess_returns), 3))
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('25th percentile returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((percentile_25_long_hpr), 3))
    print('short holding period return:', round((percentile_25_short_hpr), 3))
    print('long annualized holding period return:', round((percentile_25_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((percentile_25_short_annaulized_hpr), 3))
    print('long excess return:', round((percentile_25_long_excess_returns), 3))
    print('short excess return:', round((percentile_25_short_excess_returns), 3))
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('75th percentile returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((percentile_75_long_hpr), 3))
    print('short holding period return:', round((percentile_75_short_hpr), 3))
    print('long annualized holding period return:', round((percentile_75_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((percentile_75_short_annaulized_hpr), 3))
    print('long excess return:', round((percentile_75_long_excess_returns), 3))
    print('short excess return:', round((percentile_75_short_excess_returns), 3))
    print('\n')
    print('--------------------------------------------------------------------------------')
    print('90th percentile returns')
    print('--------------------------------------------------------------------------------')
    print('long holding period return:', round((percentile_90_long_hpr), 3))
    print('short holding period return:', round((percentile_90_short_hpr), 3))
    print('long annualized holding period return:', round((percentile_90_long_annaulized_hpr), 3))
    print('short annualized holding period return:', round((percentile_90_short_annaulized_hpr), 3))
    print('long excess return:', round((percentile_90_long_excess_returns), 3))
    print('short excess return:', round((percentile_90_short_excess_returns), 3))
    print('\n')
    return None

In [43]:
# # high book-to-market portfolio

# replace 'N/A' values with nan in stock_price_df
stock_price_df = stock_price_df.replace('N/A', np.nan)

print('HIGH BOOK-TO-MARKET PORTFOLIO (ALL F-SCORES)')
print('\n')
portfolio_analysis_statistics(stock_price_df.dropna())

HIGH BOOK-TO-MARKET PORTFOLIO (ALL F-SCORES)


number of observations: 1453


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.234
short holding period return: 0.353
long annualized holding period return: 1013.957
short annualized holding period return: 0.845
long excess return: 1013.853
short excess return: 0.935


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.012
short holding period return: 0.013
long annualized holding period return: -0.015
short annualized holding period return: 0.015
long excess return: -0.118
short excess return: 0.103


--------------------------------------------------------------------------------
10th percentile returns
--------------------------

In [44]:
# F-SCORE = 0 portfolio 
print('F-SCORE = 0 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==0])

F-SCORE = 0 PORTFOLIO


number of observations: 1


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: -0.298
short holding period return: 0.425
long annualized holding period return: -0.298
short annualized holding period return: 0.425
long excess return: -0.404
short excess return: 0.521


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.298
short holding period return: 0.425
long annualized holding period return: -0.298
short annualized holding period return: 0.425
long excess return: -0.404
short excess return: 0.521


--------------------------------------------------------------------------------
10th percentile returns
-------------------------------------------------------

In [45]:
# F-SCORE = 1 portfolio 
print('F-SCORE = 1 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==1])

F-SCORE = 1 PORTFOLIO


number of observations: 7


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: -0.03
short holding period return: 0.685
long annualized holding period return: -0.13
short annualized holding period return: 1.026
long excess return: -0.214
short excess return: 1.1


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.105
short holding period return: 0.117
long annualized holding period return: -0.342
short annualized holding period return: 0.521
long excess return: -0.447
short excess return: 0.635


--------------------------------------------------------------------------------
10th percentile returns
-----------------------------------------------------------

In [46]:
# F-SCORE = 2 portfolio 
print('F-SCORE = 2 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==2])

F-SCORE = 2 PORTFOLIO


number of observations: 70


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.338
short holding period return: 0.241
long annualized holding period return: 0.713
short annualized holding period return: 0.794
long excess return: 0.623
short excess return: 0.872


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: 0.037
short holding period return: -0.036
long annualized holding period return: 0.037
short annualized holding period return: -0.036
long excess return: -0.045
short excess return: 0.041


--------------------------------------------------------------------------------
10th percentile returns
---------------------------------------------------------

In [47]:
# F-SCORE = 3 portfolio 
print('F-SCORE = 3 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==3])

F-SCORE = 3 PORTFOLIO


number of observations: 184


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.448
short holding period return: 0.713
long annualized holding period return: 3.601
short annualized holding period return: 1.106
long excess return: 3.471
short excess return: 1.214


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: 0.052
short holding period return: -0.049
long annualized holding period return: 0.052
short annualized holding period return: -0.049
long excess return: -0.118
short excess return: 0.092


--------------------------------------------------------------------------------
10th percentile returns
--------------------------------------------------------

In [48]:
# F-SCORE = 4 portfolio 
print('F-SCORE = 4 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==4])

F-SCORE = 4 PORTFOLIO


number of observations: 300


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.324
short holding period return: 0.407
long annualized holding period return: 4907.234
short annualized holding period return: 1.273
long excess return: 4907.126
short excess return: 1.366


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.028
short holding period return: 0.029
long annualized holding period return: -0.035
short annualized holding period return: 0.037
long excess return: -0.131
short excess return: 0.122


--------------------------------------------------------------------------------
10th percentile returns
--------------------------------------------------

In [49]:
# F-SCORE = 5 portfolio 
print('F-SCORE = 5 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==5])

F-SCORE = 5 PORTFOLIO


number of observations: 354


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.221
short holding period return: 0.212
long annualized holding period return: 0.997
short annualized holding period return: 0.601
long excess return: 0.894
short excess return: 0.689


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: 0.0
short holding period return: -0.0
long annualized holding period return: 0.0
short annualized holding period return: -0.0
long excess return: -0.1
short excess return: 0.089


--------------------------------------------------------------------------------
10th percentile returns
------------------------------------------------------------------

In [50]:
# F-SCORE = 6 portfolio 
print('F-SCORE = 6 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==6])

F-SCORE = 6 PORTFOLIO


number of observations: 285


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.203
short holding period return: 0.38
long annualized holding period return: 0.195
short annualized holding period return: 0.646
long excess return: 0.102
short excess return: 0.727


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: 0.002
short holding period return: -0.002
long annualized holding period return: 0.002
short annualized holding period return: -0.002
long excess return: -0.094
short excess return: 0.078


--------------------------------------------------------------------------------
10th percentile returns
---------------------------------------------------------

In [51]:
# F-SCORE = 7 portfolio 
print('F-SCORE = 7 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==7])

F-SCORE = 7 PORTFOLIO


number of observations: 175


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.012
short holding period return: 0.244
long annualized holding period return: -0.023
short annualized holding period return: 0.936
long excess return: -0.126
short excess return: 1.024


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.063
short holding period return: 0.067
long annualized holding period return: -0.081
short annualized holding period return: 0.089
long excess return: -0.188
short excess return: 0.186


--------------------------------------------------------------------------------
10th percentile returns
------------------------------------------------------

In [52]:
# F-SCORE = 8 portfolio 
print('F-SCORE = 8 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==8])

F-SCORE = 8 PORTFOLIO


number of observations: 70


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: -0.032
short holding period return: 0.178
long annualized holding period return: -0.061
short annualized holding period return: 0.309
long excess return: -0.151
short excess return: 0.389


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.061
short holding period return: 0.065
long annualized holding period return: -0.067
short annualized holding period return: 0.072
long excess return: -0.16
short excess return: 0.178


--------------------------------------------------------------------------------
10th percentile returns
-------------------------------------------------------

In [53]:
# F-SCORE = 9 portfolio 
print('F-SCORE = 9 PORTFOLIO')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[stock_price_df['F_SCORE']==9])

F-SCORE = 9 PORTFOLIO


number of observations: 9


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.118
short holding period return: -0.021
long annualized holding period return: 0.149
short annualized holding period return: -0.038
long excess return: 0.001
short excess return: 0.089


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: 0.228
short holding period return: -0.185
long annualized holding period return: 0.228
short annualized holding period return: -0.185
long excess return: 0.099
short excess return: -0.072


--------------------------------------------------------------------------------
10th percentile returns
--------------------------------------------------------

In [54]:
# high F-SCORE portfolio (F_SCORE of 8 or 9)
print('HIGH F-SCORE PORTFOLIO (F-SCORE = 8 and F-SCORE = 9)')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[(stock_price_df['F_SCORE']==8) | (stock_price_df['F_SCORE']==9)])

HIGH F-SCORE PORTFOLIO (F-SCORE = 8 and F-SCORE = 9)


number of observations: 79


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: -0.015
short holding period return: 0.155
long annualized holding period return: -0.037
short annualized holding period return: 0.269
long excess return: -0.134
short excess return: 0.355


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.036
short holding period return: 0.037
long annualized holding period return: -0.036
short annualized holding period return: 0.037
long excess return: -0.149
short excess return: 0.144


--------------------------------------------------------------------------------
10th percentile returns
-----------------------

In [55]:
# low F-SCORE portfolio (F_SCORE of 0 or 1)
print('LOW F-SCORE PORTFOLIO (F-SCORE = 0 and F-SCORE = 1)')
print('\n')
portfolio_analysis_statistics(stock_price_df.loc[(stock_price_df['F_SCORE']==0) | (stock_price_df['F_SCORE']==1)])

LOW F-SCORE PORTFOLIO (F-SCORE = 0 and F-SCORE = 1)


number of observations: 8


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: -0.064
short holding period return: 0.652
long annualized holding period return: -0.151
short annualized holding period return: 0.951
long excess return: -0.237
short excess return: 1.028


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.201
short holding period return: 0.271
long annualized holding period return: -0.32
short annualized holding period return: 0.473
long excess return: -0.426
short excess return: 0.578


--------------------------------------------------------------------------------
10th percentile returns
--------------------------

In [58]:
# go long on high F-SCORE portfolio and short low F-SCORE portfolio
high_low_f_score_portfolio_df = stock_price_df.loc[(stock_price_df['F_SCORE']<2) | (stock_price_df['F_SCORE']>7)]

# define variables to store returns
hpr = 0
annualized_hpr = 0
excess_return = 0

for idx, row in high_low_f_score_portfolio_df.iterrows():
    if row['F_SCORE']==0 or row['F_SCORE']==1:
        hpr = hpr + row['Short_Holding_Period_Return_(%)']
        annualized_hpr = annualized_hpr + row['Short_Annualized_Holding_Period_Return_(%)']
        excess_return = excess_return + row['Short_Excess_Returns_(%)']        
    else:
        hpr = hpr + row['Long_Holding_Period_Return_(%)']
        annualized_hpr = annualized_hpr + row['Long_Annualized_Holding_Period_Return_(%)']
        excess_return = excess_return + row['Long_Excess_Returns_(%)']    

# calculate averages of returns
average_hpr = hpr / high_low_f_score_portfolio_df.shape[0]
average_annualized_hpr = annualized_hpr / high_low_f_score_portfolio_df.shape[0]
average_excess_return = excess_return / high_low_f_score_portfolio_df.shape[0]

print('LONG HIGH F-SCORE PORTFOLIO (F-SCORE = 8 and F-SCORE = 9) and SHORT LOW F-SCORE PORTFOLIO (F-SCORE = 0 and F-SCORE = 1)')
print('\n')
print('---------------------------------------------------------------------------------------')
print('average holding period return:', round(average_hpr, 3))
print('average annaulized holding period return:', round(average_annualized_hpr, 3))
print('average excess return:', round(average_excess_return, 3))

LONG HIGH F-SCORE PORTFOLIO (F-SCORE = 8 and F-SCORE = 9) and SHORT LOW F-SCORE PORTFOLIO (F-SCORE = 0 and F-SCORE = 1)


---------------------------------------------------------------------------------------
average holding period return: 0.046
average annaulized holding period return: 0.054
average excess return: -0.027


In [77]:
# all b/m companies returns without outlier
stock_price_no_outlier_df = stock_price_df.loc[stock_price_df['Long_Annualized_Holding_Period_Return_(%)']!=1472071.0572556225]
print('HIGH BOOK-TO-MARKET PORTFOLIO EXCLUDING OUTLIER (ALL F-SCORES)')
print('\n')
portfolio_analysis_statistics(stock_price_no_outlier_df.dropna())

HIGH BOOK-TO-MARKET PORTFOLIO EXCLUDING OUTLIER (ALL F-SCORES)


number of observations: 1452


--------------------------------------------------------------------------------
mean returns
--------------------------------------------------------------------------------
long holding period return: 0.234
short holding period return: 0.353
long annualized holding period return: 0.832
short annualized holding period return: 0.847
long excess return: 0.728
short excess return: 0.936


--------------------------------------------------------------------------------
median returns
--------------------------------------------------------------------------------
long holding period return: -0.013
short holding period return: 0.013
long annualized holding period return: -0.015
short annualized holding period return: 0.015
long excess return: -0.118
short excess return: 0.103


--------------------------------------------------------------------------------
10th percentile returns
--------------

## Save stock_price_df to CSV File

In [38]:
# save stock_price_df to csv file
stock_price_df.to_csv('stock_price_df.csv', index=False)

## Close WRDS Data Connection

In [39]:
# close connection to wrds
db.close()